In [1]:
import os
import sys
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import style
style.use('ggplot')
import seaborn as sns
import tqdm
import random

from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

!pip install xgboost
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance, plot_tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.model_selection import RepeatedStratifiedKFold

import warnings
warnings.filterwarnings("ignore")

data_dir = '/mnt/elice/dataset'

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# 재현성 확보를 위해 시드값을 고정합니다.
seed = 42
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [3]:
# train_df: serial number, timestamp, X1, X2~18, Y
train_df = pd.read_csv(os.path.join(data_dir, "train.csv"), index_col='Serial Number')
# test_x: serial number, timestamp, X1, X2~18
test_x = pd.read_csv(os.path.join(data_dir, "test_x.csv"), index_col='Serial Number')

''' timestamp 열 형식 바꾸기 '''
train_df['TIMESTAMP'] = pd.to_datetime(train_df['TIMESTAMP'])
train_df['TIMESTAMP'] = train_df['TIMESTAMP'].map(lambda t: t.strftime('%Y-%m-%d %H:%M'))
test_x['TIMESTAMP'] = pd.to_datetime(test_x['TIMESTAMP'])
test_x['TIMESTAMP'] = test_x['TIMESTAMP'].map(lambda t: t.strftime('%Y-%m-%d %H:%M'))

''' 컬럼 키 추출 '''
serial_key = train_df.index.name
date_time_key = list(train_df.columns)[0]
feature_keys = list(train_df.columns)[2:-1]
target_key = list(train_df.columns)[-1]

# train_x: serial number, timestamp, X1, X2~18
train_x = train_df.drop(columns='Y')
# train_y: serial_number, Y
train_y = pd.read_csv(os.path.join(data_dir, "train_y.csv"), index_col='Serial Number')

# 데이터 전처리

## 장비 이름을 나타내는 X1 변수를 제거

In [4]:
train_x.drop(columns='X1', inplace=True)
test_x.drop(columns='X1', inplace=True)

## `StandardScaler` 를 활용해 표준화 

In [5]:
scaler = StandardScaler()

train_x[feature_keys] = scaler.fit_transform(train_x[feature_keys])
test_x[feature_keys] = scaler.transform(test_x[feature_keys])

## Serial Number를 기준으로 분리

In [6]:
# 데이터를 index가 같은 컬럼 별로 분리해서 리스트에 저장
# group: (그룹 이름, 그룹 데이터프레임) => group[1]: 그룹 데이터프레임
train_x_by_serial = [group[1] for group in train_x.groupby(train_x.index)]
test_x_by_serial = [group[1] for group in test_x.groupby(test_x.index)]

# TIMESTAMP 값을 기준으로 정렬
train_x_by_serial = [group.sort_values('TIMESTAMP') for group in train_x_by_serial]
test_x_by_serial = [group.sort_values('TIMESTAMP') for group in test_x_by_serial]

In [17]:
train_x_by_serial[2]

,TIMESTAMP,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
Serial Number,,,,,,,,,,,,,,,,,,
48,2020-02-09 08:01,0.237359,0.176644,1.060111,1.067976,1.258588,-0.079582,0.845695,0.629975,0.059574,-1.734490,-0.465737,-0.012637,-0.035175,-0.464819,-0.010677,-0.011952,-0.013366
48,2020-02-10 08:16,0.237359,0.176644,1.060111,1.067976,1.258588,-0.079582,0.849884,0.633251,0.059664,-1.684896,-0.410967,-0.012621,-0.035175,-0.409868,-0.010677,-0.011952,-0.012643
48,2020-02-11 08:31,0.237359,0.176644,1.060111,1.067976,1.258588,-0.079582,0.854143,0.636589,0.059844,-1.635302,-0.315119,-0.012613,-0.035175,-0.313703,-0.010677,-0.011952,-0.012282
48,2020-02-12 08:46,0.237359,0.176644,1.060111,1.067976,1.258588,-0.079582,0.858803,0.640521,0.060115,-1.583642,-0.219271,-0.012589,-0.035175,-0.217539,-0.010677,-0.011952,-0.011199
48,2020-02-13 09:01,0.237359,0.176644,1.060111,1.067976,1.258588,-0.079582,0.863227,0.644140,0.060295,-1.534048,-0.127987,-0.012589,-0.035175,-0.125954,-0.010677,-0.011952,-0.011199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,2020-04-14 00:01,0.237359,0.176644,1.407310,1.067976,1.258588,-0.079582,1.133848,0.867393,0.078570,1.480844,4.029984,-0.012373,-0.035175,4.045745,-0.010677,-0.011952,-0.001446
48,2020-04-15 00:16,0.237359,0.176644,1.407310,1.067976,1.258588,-0.079582,1.138296,0.870981,0.078840,1.530438,4.198859,-0.012345,-0.035175,4.215177,-0.010677,-0.011952,-0.000362
48,2020-04-16 00:31,0.237359,0.176644,1.407310,1.067976,1.258588,-0.079582,1.142649,0.874445,0.079110,1.580032,4.317527,-0.012345,-0.035175,4.334238,-0.010677,-0.011952,-0.000362


In [18]:
train_x_by_serial[2].index.unique()

Int64Index([48], dtype='int64', name='Serial Number')

In [19]:
train_x_by_serial[2].index.unique().item()

48

In [20]:
train_y.loc[54509]

Y    0
Name: 54509, dtype: int64

## train, valid 분할

In [7]:
def train_test_split(Xs, ys, test_ratio=0.2):
    ''' 각 (x, y) 쌍을 label 별로 딕셔너리에 저장 '''
    data_per_label = {}

    for x, y in zip(Xs, ys):
        label = y
        if label not in data_per_label:
            data_per_label[label] = []
        # key: label, value: 해당 label에 해당하는 (x, y) 쌍의 리스트
        data_per_label[label].append((x, y))

    train = []
    test = []

    for label in data_per_label:
        # label에 해당하는 데이터 가져온다
        data = data_per_label[label]
        # 테스트 데이터의 개수
        n_test = int(len(data) * test_ratio)
        test += data[:n_test]
        train += data[n_test:]

    X_train, y_train = zip(*train)
    X_test, y_test = zip(*test)

    return X_train, X_test, y_train, y_test

In [8]:
X_train, X_val, y_train, y_val = train_test_split(train_x_by_serial, train_y['Y'], test_ratio=0.2)

# X data에서 Timestamp를 제거합니다.
X_train = [x.drop(columns='TIMESTAMP') for x in X_train]
X_val = [x.drop(columns='TIMESTAMP') for x in X_val]
X_test = [x.drop(columns='TIMESTAMP') for x in test_x_by_serial]

print("Train Data의 개수 :", len(X_train))
print("Validation Data의 개수 :", len(X_val))
print("Test Data의 개수 :", len(X_test))

Train Data의 개수 : 6618
Validation Data의 개수 : 1654
Test Data의 개수 : 2069


## 머신러닝 모델에 적용하기 위해 학습, 검증, 테스트용 데이터를 각각 하나의 numpy array로 합칩니다.

In [9]:
def align_data(data, series_length):
    # X2~18열에 해당하는 시계열 데이터 값만 추출
    data_features = [x[feature_keys] for x in data]
    len_data = len(data_features)
    length_aligned_X = []
    for x in data_features:
        # 시계열 데이터 길이가 series_length 이상이면 > 뒷부분 잘라냄
        if len(x) >= series_length:
            length_aligned_X.append(x[:series_length])
        # 시계열 데이터 길이가 series_length보다 작으면 > 마지막 행을 반복하여 길이를 맞춤
        else:
            length_aligned_X.append(x.append([x.iloc[-1]] * (series_length - len(x))))
    return np.array(length_aligned_X).reshape(len_data, -1)

In [10]:
series_length = 67

X_train = align_data(X_train, series_length)
X_val = align_data(X_val, series_length)
X_test = align_data(X_test, series_length)

In [11]:
y_train = np.array(y_train)
y_val = np.array(y_val)

# 모델 학습

In [16]:
# 완전 기본 모델!
model = XGBClassifier(random_state = 42)
model.fit(X_train, y_train)

# 검증 데이터에 대한 예측
y_val_pred = model.predict(X_val)
# 검증 데이터에 대한 F1 점수 계산
f1_val = f1_score(y_val, y_val_pred, average='macro')
print('Validation F1 score = %.3f' % f1_val)

Validation F1 score = 0.890


# 모델 평가 및 결과 저장


In [17]:
submission = pd.read_csv(os.path.join(data_dir, "test_y.csv"), index_col='Serial Number')

# 테스트 데이터에 대해 예측하고, 결과를 저장합니다.
y_test_pred = model.predict(X_test)
submission["Y"] = y_test_pred
submission.to_csv("submission.csv", index_label='Serial Number')

### 결과 검증

`submission.csv` 파일을 다시 불러와 올바르게 값을 채웠는지 다시 한번 확인합니다.

In [18]:
submission = pd.read_csv("submission.csv", index_col='Serial Number')
submission

,Y
Serial Number,
100122I,1
100368G,0
101403L,0
101426G,0
101505B,0
...,...
997719U,0
998737L,0
999308S,0


In [19]:
# 예측 결과 중 1의 비율을 계산하고, 학습용 데이터의 비율과 비교합니다.
print(submission["Y"].mean())
print(train_y.mean())

0.17158047365877235
Y    0.146518
dtype: float64


### 제출

우측 상단의 제출 버튼을 눌러, `competition.ipynb` 파일과 `submission.csv` 파일을 제출합니다.